In [2]:
import csv
import time
import itertools as it
import numpy as np
import math
import random

distance_matrix = [] #matrix with cities and distances
cities = [] #list with city names
data = [] #data from the file

###functions common to all three algorithms

#load data into a numpy array
def load_data(file, number_of_cities):
    with open(file, "r") as f:
        data_from_file = np.array(list(csv.reader(f, delimiter=';'))) #convert list to numpy array
        
    global data
    data = data_from_file
    #print data[:7,:6]

    global cities
    cities = data[:1] # array of all cities
    
    print "Data loaded!"

def initialize_matrix(number_of_cities):
    row = number_of_cities + 1
    column = number_of_cities
    global distance_matrix
    distance_matrix = np.array(data[:row,:column])
    #print distance_matrix
    
    global cities
    cities = distance_matrix[0] #names of the cities
    
#function return the name of the city behind the index
def get_city_name(i):
    return cities[i]

#function returns distance between two cities
def get_distance_between_two_cities(start, stop):
    return float(distance_matrix[start+1,stop])

def calculate_total_distance(order_of_visits):
    route = "" #string for descibing the route with city names
    route_index = [] #list with order of city indexes to describer route
    total_distance = 0
    order = order_of_visits + [order_of_visits[0]]
        
    for i in xrange(len(order)-1):
        start_city_index = order[i] #index of start city
        stop_city_index = order[i+1] #index of stop city
        start_city = get_city_name(start_city_index) #name of start city
        stop_city = get_city_name(stop_city_index) #name of stop city
        #print "start city:", start_city
        #print "stop_city", stop_city
        distance = get_distance_between_two_cities(start_city_index, stop_city_index) #get distance between cities
        #print "distance", distance
        total_distance += distance #add current distance to the total
        route = route + start_city + " " #add current city to the route

        route_index.append(i)
        
    return round(total_distance, 2), route    


###########################
#### EXHAUSTIVE SEARCH ####
###########################
#takes number of cities as a parameter and checks EVERY single permutation. 
#The permutation with shortest travelled distance is the preferred route
def exhaustive_search(number_of_cities):
    number_of_all_routes = math.factorial(number_of_cities)
    
    initialize_matrix(number_of_cities)    
    
    start = time.time()
    counter = 0
    min_distance = 9999999
    min_route = []
    city_index = range(number_of_cities)

    #loop through all permutations
    #permutations function takes in two parameters: list of candidates and number of elements in each permutation.
    #For example: permutations([1,2,3,4], 2) -> (1,2), (1,3)...
    for l in list(it.permutations(city_index, number_of_cities)):
        counter += 1
        distance, route = calculate_total_distance(list(l))
        if distance < min_distance:
            min_distance = distance
            min_route = route 

    end = time.time()
    run_time = round((end - start), 8)
    
    print "EXHAUSTIVE SEARCH. Number of cities:", number_of_cities
    print "Shortest route: %s\nRoute: %s\nTime:%s seconds" % (min_distance, min_route, run_time)

    

#######################
#### HILL CLIMBING ####
#######################
def hill_climbing(number_of_cities, number_of_swaps):
    #initialize_dataset(number_of_cities)
    
    list_of_city_index = range(number_of_cities) #create list of city indexes
    #print list_of_city_index
    
    list_of_city_index[0], list_of_city_index[1] = list_of_city_index[1], list_of_city_index[0]
    #print list_of_city_index
    
    np.random.shuffle(list_of_city_index) #random shuffle to get initial solution
    inital_solution = list_of_city_index #prepare initial solution
    #print inital_solution
    min_distance, optimal_route = calculate_total_distance(inital_solution)
    #print min_distance, optimal_route
    
    
    #print "Start swapping:"
    for i in range(1, number_of_swaps+1):
        #print i
        first_city = np.random.randint(0, number_of_cities)
        second_city = np.random.randint(0, number_of_cities)
        #print first_city, "<->", second_city
        #print "Swap cities: %s <-> %s" % (list_of_city_index[first_city], list_of_city_index[second_city])
        list_of_city_index[first_city], list_of_city_index[second_city] = list_of_city_index[second_city], list_of_city_index[first_city]
        #print list_of_city_index
        temp_distance, temp_route = calculate_total_distance(list_of_city_index)
        #print temp_distance, temp_route
        if (temp_distance < min_distance):
            min_distance = temp_distance
            optimal_route = temp_route
            
    
    return min_distance, optimal_route

def run_hill_climbing(cities_number, swaps_number):
    import time
    list_of_results_distance = [] #list for distances
    list_of_results_route = [] #list for lists of routes

    start2 = time.time()
    
    #first 10 cities with 1000 swaps
    for i in xrange(1,21): #run 20 times 
        #print i
        distance, route = hill_climbing(cities_number, swaps_number)
        list_of_results_distance.append(distance)
        list_of_results_route.append(route)
        
    end2 = time.time()
    run_time = round((end2 - start2), 4)
    
    
    min_route = min(list_of_results_distance)
    max_route = max(list_of_results_distance)
    list_distance_np = np.array(list_of_results_distance)
    mean_route = np.mean(list_distance_np)
    std_dev = round(np.std(list_distance_np, ddof=1), 4)
    
    index_of_min_route = list_of_results_distance.index(min_route) #get index of the shortest distance
    min_route_cities = list_of_results_route[index_of_min_route] #city to city route that has shortest distance
    
    print "HILL CLIMBING. Number of cities: %s, number of swaps: %s" % (cities_number, swaps_number)
    print "Shortest route: %s\nLongest route: %s\nMean distance: %s\nStandard deviation: %s" % (min_route, max_route, mean_route, std_dev)
    print "Route travelled for this distance: %s" % (min_route_cities)
    print "Time needed: %s" % (run_time)
    
#function runs a comparison check for 10 cities between exhaustive search and hill climbing
def compare_es_and_hc_with_10_cities(number_of_swaps):
    print "Calculating best routes for 10 cities...\n"
    number_of_cities=10
    exhaustive_search(number_of_cities)
    
    initialize_matrix(number_of_cities)
    run_hill_climbing(number_of_cities, number_of_swaps)

#function runs hill climbing algorithm with 2 parameters - number of cities and number of swaps    
def test_hill_climbing(number_of_cities, number_of_swaps):
    initialize_matrix(number_of_cities)
    run_hill_climbing(number_of_cities, number_of_swaps)
    

############    
### RUN ####
############

####load data
load_data("/user/marko/emne/inf-3490/european_cities.csv", 6)


####exhaustive search with first 6 cities
#exhaustive_search(6)
    
    
####function runs comparison for 10 cities between exhaustive search and hill climbing.
####the input parameter is number of swaps for the hill climbing algorithm
####Can take up to 1 minute to run!
#compare_es_and_hc_with_10_cities(1000)


####run hill climbing
#### 1. parameter: number of cities, 2. parameter: number of swaps
test_hill_climbing(24, 10000)



Data loaded!
HILL CLIMBING. Number of cities: 24, number of swaps: 10000
Shortest route: 21312.0
Longest route: 23553.21
Mean distance: 22418.8055
Standard deviation: 558.458
Route travelled for this distance: Berlin Budapest Munich Prague Vienna Rome Copenhagen Brussels Milan Madrid Paris Dublin Hamburg Barcelona Istanbul Sofia Belgrade Bucharest Kiev Warsaw Saint Petersburg Moscow Stockholm London 
Time needed: 5.9993
